In [1]:
# Built-in imports
import os
import sys
from typing import List, Dict
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# GravyFlow import, again adding the grandparent directory to the path:

# Get the absolute path of the grandparent directory (two levels up from the current directory)
parent_dir : Path = os.path.abspath('../../')

# Add the grandparent directory to sys.path if it's not already there.
# This is necessary to import GravyFlow if it's located in the grandparent directory.
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import the GravyFlow module.
import gravyflow as gf

2024-01-31 17:41:50.193559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 17:41:52.071061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env()

INFO:root:TensorFlow version: 2.12.1, CUDA version: 11.8
2024-01-31 17:42:01.153560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)

phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

2024-01-31 17:42:02.708461: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55d94d949d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 17:42:02.708500: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-01-31 17:42:02.744190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-01-31 17:42:02.766946: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


## Projecting Injections onto Multiple Detectors

In [4]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.V1])

2024-01-31 17:42:03.162469: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-31 17:42:03.287431: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [6]:
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0],
        right_ascension=0.0,
        declination=0.0,
        polarization=0.0
    )

2024-01-31 17:42:31.131333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_4789}}{{node PartitionedCall}}]]


In [7]:
projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

In [8]:
# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]
projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-01-31 17:42:37.760237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_4789}}{{node PartitionedCall}}]]


Loading BokehJS ...

In [9]:
# Adding Custom Detectors:

# Create from dictionary:
network = gf.Network({
    "longitude_radians" : [np.pi/2, np.pi/4], 
    "latitude_radians" : [-np.pi/4, np.pi/6],
    "y_angle_radians" : [(2*np.pi/3), np.pi], 
    "x_angle_radians" : None, 
    "height_meters" : [0.0, 0.0],
    "x_length_meters" : [4000.0, 10000.0],
    "y_length_meters" : [4000.0, 10000.0]
})

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-01-31 17:42:43.471286: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_5331}}{{node PartitionedCall}}]]


Loading BokehJS ...

## Load Detector From Config

In [10]:
# Define injection directory path:
example_network_directory : Path =  Path("./example_configs/example_network.json")

network = gf.Network.load(example_network_directory)

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Randomise Detectors

In [11]:
# Adding Custom Detectors:

# FIX: This is not uniform distribution across a sphere:
longitude_distribution_radians : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=2.0*np.pi, 
    type_=gf.DistributionType.UNIFORM
)
latitude_distribution_msun : gf.Distribution = gf.Distribution(
    min_=-np.pi/2.0, 
    max_=np.pi/2.0, 
    type_=gf.DistributionType.UNIFORM
)
y_angle_distribution_radians: gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=2*np.pi, 
    type_=gf.DistributionType.UNIFORM
)
x_angle_distribution_radians: gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=2*np.pi, 
    type_=gf.DistributionType.UNIFORM
)
height_distribution_meters: gf.Distribution = gf.Distribution(
    min_=-1000, 
    max_=1000, 
    type_=gf.DistributionType.UNIFORM
)
x_length_meters : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=10000, 
    type_=gf.DistributionType.UNIFORM
)
y_length_meters : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=10000, 
    type_=gf.DistributionType.UNIFORM
)

# Create from dictionary:
network = gf.Network({
    "num_detectors" : 3,
    "longitude_radians" : longitude_distribution_radians, 
    "latitude_radians" : latitude_distribution_msun,
    "y_angle_radians" : y_angle_distribution_radians, 
    "x_angle_radians" : x_angle_distribution_radians, 
    "height_meters" : height_distribution_meters,
    "x_length_meters" : x_length_meters,
    "y_length_meters" : y_length_meters
})

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Incoherent Injections

In [14]:
wnb_generator_a : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50.0,
    max_frequency_hertz=100.0
)
phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)

incoherent_generator : gf.InjectionGenerator  = gf.IncoherentGenerator(
    [wnb_generator_a, phenom_d_generator]
)

incoherent_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(incoherent_generator)    

In [16]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1])
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    incoherent_injections, _, _ = next(
        incoherent_injection_generator(num_examples_per_batch=1)
    )        

    projected_injections : tf.Tensor = network.project_wave(
        incoherent_injections,
        right_ascension=0.0,
        declination=0.0,
        polarization=0.0
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-01-31 17:44:42.026805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_19110}}{{node PartitionedCall}}]]


Loading BokehJS ...